# Costruzioni dataset

In [2]:
import os
import sentencepiece as spm
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import xml.dom.minidom as minidom
import spacy

## Variabili

In [5]:
lingua_dataset = 'it-PE'
tipo_dataset_generato = 'TCO'
#random_number = str(np.random.randint(0, 100000))
flag_shape = True
flag_shape_type = True
flag_size = True

## Dataset per RDF2Text

### Train e dev set

In [7]:
dataset_types = ["train"] 
triple_numbers = ["1", "2", "3", "4", "5", "6", "7"]
dataset = []

i = 0
max_length = 0
for dataset_type in dataset_types:
    for triple_number in triple_numbers:
        path = "webnlg\\it-PE\\" + dataset_type + "\\" + triple_number + "triples"
        file_names = []
        for file_name in os.listdir(path):
            if os.path.isfile(os.path.join(path, file_name)):
                i += 1
                url = os.path.join(path, file_name)
                tree = ET.parse(url)
                root = tree.getroot()

                shape = ''
                shape_type = ''
                size = 1
                for entry in root.iter('entry'):
                    if flag_shape:
                        shape = entry.get('shape')

                    if flag_shape_type:
                        shape_type = entry.get('shape_type')

                    if flag_size:
                        size = entry.get('size')

                    mts = []
                    
                    for modifiedtripleset in entry.iter('modifiedtripleset'):
                        mtriples = ""

                        if (tipo_dataset_generato == 'TC'): #senza ordinamento alfabetico
                            for mtriple in modifiedtripleset.iter('mtriple'):                            
                                #mtriples += mtriple.text.replace(" | ", " ") + " "
                                mtriples += mtriple.text.replace(" | ", " ") + " | "


                        elif (tipo_dataset_generato == 'TCO'): #con ordinamento alfabetico
                            triples = []
                            for mtriple in modifiedtripleset.iter('mtriple'):  
                                triple = mtriple.text.split(" | ")
                                triple = (triple[0], triple[1], triple[2])
                                triples.append(triple)

                            triples.sort(key=lambda x: x[1])
                            for triple in triples:
                                mtriples += triple[0] + " " + triple[1] + " " + triple[2] + " "

                        elif (tipo_dataset_generato == 'TEO'): # con ordinamento e tag
                            triples = []
                            for mtriple in modifiedtripleset.iter('mtriple'):  
                                triple = mtriple.text.split(" | ")
                                triple = (triple[0], triple[1], triple[2])
                                triples.append(triple)

                            triples.sort(key=lambda x: x[1])
                            for triple in triples:
                                mtriples += " <S> " + triple[0] + " <P> " + triple[1] + " <O> " + triple[2]
                        else:
                            break

                        mts.append(mtriples)

                    lexs = []
                    for lex in entry.iter('lex'):
                        if lex.get('lang') == lingua_dataset:
                            lexs.append(lex.text)
                            #get length of lex.text
                            if len(lex.text) > max_length:
                                max_length = len(lex.text)

                    for mt in mts:
                        for lex in lexs:
                            i += 1
                            dataset.append([mt, lex, shape, shape_type, size])

dataset = np.array(dataset)
print(max_length)

dataset_pd = pd.DataFrame(dataset, columns=['triple', 'sentence', 'shape', 'shape_type', 'size'])
dataset_pd.to_csv(random_number + "_train_" + tipo_dataset_generato + "_" + lingua_dataset + ".csv", index=False)

444


### Dev set

In [8]:
dataset_types = ["dev"] 
triple_numbers = ["1", "2", "3"]#, "4", "5", "6", "7"]
dataset = []

i = 0
for dataset_type in dataset_types:
    for triple_number in triple_numbers:
        path = "webnlg\\it-PE\\" + dataset_type + "\\" + triple_number + "triples"
        file_names = []
        for file_name in os.listdir(path):
            if os.path.isfile(os.path.join(path, file_name)):
                i += 1
                url = os.path.join(path, file_name)
                tree = ET.parse(url)
                root = tree.getroot()

                shape = ''
                shape_type = ''
                size = 0
                for entry in root.iter('entry'):
                    if flag_shape:
                        shape = entry.get('shape')

                    if flag_shape_type:
                        shape_type = entry.get('shape_type')

                    if flag_size:
                        size = entry.get('size')

                    mts = []
                    
                    for modifiedtripleset in entry.iter('modifiedtripleset'):
                        mtriples = ""
                        
                        if (tipo_dataset_generato == 'TC'): #senza ordinamento alfabetico
                            for mtriple in modifiedtripleset.iter('mtriple'):                            
                                mtriples += mtriple.text.replace(" | ", " ") + " "
                        elif (tipo_dataset_generato == 'TCO'): #con ordinamento alfabetico
                            triples = []
                            for mtriple in modifiedtripleset.iter('mtriple'):  
                                triple = mtriple.text.split(" | ")
                                triple = (triple[0], triple[1], triple[2])
                                triples.append(triple)

                            triples.sort(key=lambda x: x[1])
                            for triple in triples:
                                mtriples += triple[0] + " " + triple[1] + " " + triple[2] + " "
                        elif (tipo_dataset_generato == 'TEO'): # con ordinamento e tag
                            triples = []
                            for mtriple in modifiedtripleset.iter('mtriple'):  
                                triple = mtriple.text.split(" | ")
                                triple = (triple[0], triple[1], triple[2])
                                triples.append(triple)

                            triples.sort(key=lambda x: x[1])
                            for triple in triples:
                                mtriples += " <S> " + triple[0] + " <P> " + triple[1] + " <O> " + triple[2]
                        else:
                            break

                        mts.append(mtriples)

                    lexs = []
                    for lex in entry.iter('lex'):
                        if lex.get('lang') == lingua_dataset:
                            lexs.append(lex.text)

                    for mt in mts:
                        for lex in lexs:
                            i += 1
                            dataset.append([mt, lex, shape, shape_type, size])

dataset = np.array(dataset)

dataset_pd = pd.DataFrame(dataset, columns=['triple', 'sentence', 'shape', 'shape_type', 'size'])
dataset_pd.to_csv(random_number + "_dev_" + tipo_dataset_generato + "_" + lingua_dataset + ".csv", index=False)

### Test set

In [21]:
dataset = []

i = 0

path = "..\\..\\WebNLG\\it-PE\\test"
file_names = []
for file_name in os.listdir(path):
    if os.path.isfile(os.path.join(path, file_name)):
        i += 1
        url = os.path.join(path, file_name)
        tree = ET.parse(url)
        root = tree.getroot()

        shape = ''
        shape_type = ''
        size = 0
        for entry in root.iter('entry'):
            if flag_shape:
                shape = entry.get('shape')

            if flag_shape_type:
                shape_type = entry.get('shape_type')
            
            if flag_size:
                size = entry.get('size')

            mts = []

            for modifiedtripleset in entry.iter('modifiedtripleset'):
                mtriples = ""

                if (tipo_dataset_generato == 'TC'): #senza ordinamento alfabetico
                    for mtriple in modifiedtripleset.iter('otriple'):                            
                        #mtriples += mtriple.text.replace(" | ", " ") + " "
                        mtriples += mtriple.text.replace(" | ", " ") + " | "
                        
                elif (tipo_dataset_generato == 'TCO'): #con ordinamento alfabetico
                    triples = []

                    for mtriple in modifiedtripleset.iter('mtriple'):
                        triple = mtriple.text.split(" | ")
                        triple = (triple[0], triple[1], triple[2])
                        triples.append(triple)

                    triples.sort(key=lambda x: x[1])
                    for triple in triples:
                        mtriples += triple[0] + " " + triple[1] + " " + triple[2] + " | "


                elif (tipo_dataset_generato == 'TEO'): # con ordinamento e tag
                    triples = []
                    for mtriple in modifiedtripleset.iter('mtriple'):  
                        triple = mtriple.text.split(" | ")
                        triple = (triple[0], triple[1], triple[2])
                        triples.append(triple)

                    triples.sort(key=lambda x: x[1])
                    for triple in triples:
                        mtriples += " <S> " + triple[0] + " <P> " + triple[1] + " <O> " + triple[2]
                else:
                    break
                            
                mts.append(mtriples)

            lexs = []
            for lex in entry.iter('lex'):
                if lex.get('lang') == lingua_dataset:
                    lexs.append(lex.text)

            for mt in mts:
                for lex in lexs:
                    i += 1
                    dataset.append([mt, lex, shape, shape_type, size])

dataset = np.array(dataset)

dataset_pd = pd.DataFrame(dataset, columns=['triple', 'sentence', 'shape', 'shape_type', 'size'])
dataset_pd.to_csv("pre_splitted_test_" + tipo_dataset_generato + "_" + lingua_dataset + ".csv", index=False)

## Dataset per EliCoDe

In [7]:
triple_numbers = ["1"]#, "2", "3", "4", "5", "6", "7"]
dataset_types = ["dev"]

it = True
#it = False

sentences = []
for dataset_type in dataset_types:
    for triple_number in triple_numbers:
        path = "webnlg\\it\\" + dataset_type + "\\" + triple_number + "triples"

        file_names = []
        for file_name in os.listdir(path):
            if os.path.isfile(os.path.join(path, file_name)):
                url = os.path.join(path, file_name)
                tree = ET.parse(url)
                root = tree.getroot()

                for entry in root.iter('entry'):
                    originaltripleset = entry.find('modifiedtripleset')
                    otriple = originaltripleset.find('mtriple')

                    for lex in entry.iter('lex'):
                        if lex.get('lang') == "it":
                            sentences.append(lex.text)

print(len(sentences))
sentences

961


['Il leader di Aarhus è Jacob Bundsgaard.',
 "La lunghezza della pista dell'aeroporto di Aarhus è di 2702,0 metri.",
 "L'aeroporto di Aarhus ha una pista lunga 2702,0 metri.",
 "L'aeroporto regionale di Adirondack si trova a 507 metri sul livello del mare.",
 "L'aeroporto regionale dell'Adirondack si trova a Harrietstown, New York.",
 "L'aeroporto regionale dell'Adirondack si trova a Harrietstown, New York.",
 "L'aeroporto Adolfo Suárez Madrid-Barajas si trova a San Sebastián de los Reyes.",
 "L'aeroporto Adolfo Suarez Madrid-Barajas si trova a San Sebastian de los Reyes.",
 "L'Aeroporto Adolfo Suárez Madrid-Barajas si trova a San Sebastián de los Reyes.",
 "Il nome della pista dell'Aeroporto Adolfo Suárez Madrid-Barajas è 14L/32R.",
 "Il nome della pista dell'Aeroporto Adolfo Suárez Madrid-Barajas è 14L/32R.",
 "14L/32R è il nome della pista dell'Aeroporto Adolfo Suárez Madrid-Barajas.",
 "14R/32L è il nome della pista dell'aeroporto Adolfo Suárez di Madrid-Barajas.",
 "14R/32L è il n

In [14]:
tokenizer = spacy.load("it_core_news_sm")

dataset = []
for sentence in sentences:
    sentence = tokenizer(sentence)

    for token in sentence:
        dataset.append(token.text + " O O O")
    
    dataset.append("")

In [16]:
#save dataset into a file named test_dev_1.txt in the folder dataset-EliCoDe
with open("dataset-EliCoDe\\test_dev_1.txt", "w", encoding="utf-8") as f:
    for sentence in dataset:
        f.write(sentence + "\n")

## Creazione file traduzioni

In [3]:
dataset_types = ["train", "dev"] 
triple_numbers = ["1", "2", "3", "4", "5", "6", "7"]
dataset = []

i = 0
for dataset_type in dataset_types:
    for triple_number in triple_numbers:
        path = "webnlg\\it-PE\\" + dataset_type + "\\" + triple_number + "triples"
        file_names = []
        for file_name in os.listdir(path):
            if os.path.isfile(os.path.join(path, file_name)):
                i += 1
                url = os.path.join(path, file_name)
                tree = ET.parse(url)
                root = tree.getroot()


                for entry in root.iter('entry'):
                    lexs = []
                    for lex in entry.iter('lex'):
                        if lex.get('lang') == 'it':
                            lexs.append(lex.text)

                    for lex in lexs:
                        i += 1
                        dataset.append(lex)

dataset = np.array(dataset)

In [4]:
dataset_pd = pd.DataFrame(dataset, columns=['lex'])
dataset_pd.to_csv("mt.csv", index=False)